In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import operator
import pickle

In [2]:
from hpo_similarity import open_ontology
graph, alt_ids, obsolete_ids = open_ontology()

In [3]:
%%time
data = pd.read_excel('phenotype_original.xlsx')
g = data.groupby('#DatabaseID')
G= g['HPO_ID'].apply(lambda s: s.tolist())
OMIM_DICT = G.to_dict()

CPU times: user 5.9 s, sys: 50.1 ms, total: 5.95 s
Wall time: 6.05 s


In [4]:
%%time
graph.tally_hpo_terms(OMIM_DICT)

CPU times: user 293 ms, sys: 3.04 ms, total: 297 ms
Wall time: 297 ms


In [5]:
data1= data.head(108580)
data2= data.loc[108580:]

In [6]:
set_id = set(data["HPO_ID"])
print(len(set_id))

9357


In [7]:
def lin_score(graph, term1, term2):
    a = graph.calculate_information_content(term1)
    b = graph.calculate_information_content(term2)
    c = 2*graph.get_most_informative_ic(term1,term2)
    #print ("##@@",a,b,c)
    try:
        lin_score = c/(b + a)
    except ZeroDivisionError:
        lin_score = 0
    return lin_score

In [8]:
df_OMIM_ORPHA = pd.DataFrame(columns=['Phenotype_ID', 'Number_of_Disease', 'Information_Content','Lin_Score'])

## Immediate children of HP:0001371

In [9]:
children = graph.get_immediate_descendants('HP:0001371')
children

{'HP:0002803',
 'HP:0002828',
 'HP:0003121',
 'HP:0004631',
 'HP:0005781',
 'HP:0005876',
 'HP:0005997',
 'HP:0030044',
 'HP:0100492'}

### Information content and Number of disease of 'HP:0001371'

In [11]:
print (graph.calculate_information_content('HP:0001371'))
print (graph.get_term_count('HP:0001371'))
print (graph.get_ids_per_term('HP:0001371'))

2.4664508105562772
998
{'ORPHA:424107', 'OMIM:616801', 'OMIM:147791', 'ORPHA:457205', 'OMIM:600325', 'OMIM:251300', 'OMIM:607014', 'OMIM:224800', 'OMIM:605637', 'OMIM:608149', 'ORPHA:284160', 'OMIM:256030', 'OMIM:313420', 'ORPHA:83', 'OMIM:608013', 'OMIM:248370', 'OMIM:264090', 'ORPHA:246', 'OMIM:602398', 'ORPHA:1703', 'ORPHA:821', 'OMIM:617143', 'ORPHA:96061', 'OMIM:200980', 'ORPHA:370980', 'OMIM:154700', 'ORPHA:440713', 'OMIM:615490', 'OMIM:151050', 'OMIM:207410', 'OMIM:253700', 'OMIM:616081', 'OMIM:251240', 'ORPHA:261537', 'ORPHA:370968', 'OMIM:179613', 'ORPHA:85212', 'OMIM:610713', 'OMIM:618266', 'ORPHA:2136', 'OMIM:236500', 'OMIM:603513', 'OMIM:226650', 'ORPHA:284979', 'OMIM:617105', 'OMIM:609628', 'OMIM:147800', 'OMIM:184900', 'ORPHA:96164', 'OMIM:609308', 'ORPHA:2741', 'OMIM:607598', 'ORPHA:468699', 'ORPHA:98896', 'ORPHA:97297', 'OMIM:612138', 'OMIM:617591', 'OMIM:117000', 'OMIM:601110', 'ORPHA:1979', 'ORPHA:3063', 'OMIM:616006', 'OMIM:617468', 'OMIM:300695', 'ORPHA:3447', 'OMIM

In [12]:
i=0
for child in children:
    df_OMIM_ORPHA.loc[i,['Phenotype_ID']]= child
    #print (child)
    #print (graph.calculate_information_content(child))
    #print (graph.get_term_count(child))
    #print (graph.get_ids_per_term(child))
    #print (lin_score(child, 'HP:0001371'))
    df_OMIM_ORPHA.loc[i,['Number_of_Disease']]= graph.get_term_count(child)
    df_OMIM_ORPHA.loc[i,['Information_Content']]= graph.calculate_information_content(child)
    df_OMIM_ORPHA.loc[i,['Lin_Score']]= lin_score(graph, child, 'HP:0001371')
    i= i+1

### Information content, Number of disease and Lin Score while considering both OMIM and ORPHA

In [13]:
df_OMIM_ORPHA

,Phenotype_ID,Number_of_Disease,Information_Content,Lin_Score
0,HP:0002828,42,5.634534,0.608926
1,HP:0002803,174,4.213149,0.738503
2,HP:0003121,503,3.151614,0.878043
3,HP:0004631,7,7.426294,0.498638
4,HP:0005781,13,6.807255,0.531923
5,HP:0100492,31,5.938217,0.586924
6,HP:0005876,9,7.17498,0.511636
7,HP:0030044,417,3.339118,0.849684
8,HP:0005997,3,8.273592,0.4593


In [14]:
graph1, alt_ids, obsolete_ids = open_ontology()
g1 = data1.groupby('#DatabaseID')
G1= g1['HPO_ID'].apply(lambda s: s.tolist())
OMIM_DICT1 = G1.to_dict()
graph1.tally_hpo_terms(OMIM_DICT1)

In [15]:
df_OMIM = pd.DataFrame(columns=['Phenotype_ID', 'Number_of_Disease', 'Information_Content','Lin_Score'])
i=0
for child in children:
    df_OMIM.loc[i,['Phenotype_ID']]= child
    #print (child)
    #print (graph1.calculate_information_content(child))
    #print (graph1.get_term_count(child))
    #print (graph.get_ids_per_term(child))
    #print (lin_score(graph1, child, 'HP:0001371'))
    #print("\n")
    df_OMIM.loc[i,['Number_of_Disease']]= graph1.get_term_count(child)
    df_OMIM.loc[i,['Information_Content']]= graph1.calculate_information_content(child)
    df_OMIM.loc[i,['Lin_Score']]= lin_score(graph1, child, 'HP:0001371')
    i= i+1

### Information content, Number of disease and Lin Score while considering only OMIM

In [16]:
df_OMIM

,Phenotype_ID,Number_of_Disease,Information_Content,Lin_Score
0,HP:0002828,10,6.659422,0.532703
1,HP:0002803,114,4.225809,0.722609
2,HP:0003121,235,3.502422,0.808258
3,HP:0004631,2,8.26886,0.453827
4,HP:0005781,3,7.863395,0.471412
5,HP:0100492,16,6.189418,0.561187
6,HP:0005876,3,7.863395,0.471412
7,HP:0030044,192,3.704512,0.782352
8,HP:0005997,3,7.863395,0.471412
